In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 学習用モデルのインポート
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression, RidgeCV
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.cross_decomposition import PLSRegression

c:\Users\tixt1\anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [3]:
# 評価指標
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

data = pd.read_csv('train.csv')
data.head()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
mini_data = data[['GrLivArea','YearBuilt','SalePrice']]

In [ ]:
# 使用するデータセットの可視化
sns.pairplot(mini_data);

In [ ]:
# 対数変換
log_min_data = mini_data.apply(np.log1p)

In [ ]:
# 対数変換後のデータセットの可視化
sns.pairplot(log_min_data);

In [ ]:
# 説明変数X、目的変数y
X = log_min_data[['GrLivArea','YearBuilt']].values
y = log_min_data['SalePrice'].values

In [ ]:
# 1つのモデルでトレーニング、推定を行う
# Cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 単一モデル
model = [LinearRegression(),
         SVR(),
         DecisionTreeRegressor()]


# Cross-validation
kf = KFold(n_splits=5,random_state=None, shuffle=False)

# 5つのパートで検証されたメトリクスMSE
for regr in model:
    result = -cross_val_score(regr,X,y, cv = kf, scoring = "neg_mean_squared_error")
    result_mean = np.mean(result)
    
    # print('CV_MSE:',result)
    print('CV_MSE_MEAN:{:.3f}'.format(result_mean),'MODEL:',str(regr))

In [ ]:
# トレーニングデータと検証データに分割する（ホールドアウト法）
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = \
train_test_split(X,y,train_size=0.8,random_state=None)

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_valid:',X_valid.shape)
print('y_valid:',y_valid.shape)

In [ ]:
# ブレンディング 1
regr1 = LinearRegression().fit(X_train,y_train)
regr2 = SVR().fit(X_train,y_train)
regr3 = DecisionTreeRegressor().fit(X_train,y_train)

y_pred1 = regr1.predict(X_valid)
y_pred2 = regr2.predict(X_valid)
y_pred3 = regr3.predict(X_valid)


# 推定値の平均値
y_pred_blend1 = np.mean([y_pred1,y_pred2,y_pred3],axis=0)
#print(y_pred)

# 評価
mse = mean_squared_error(y_valid,y_pred_blend1)
rmse = np.sqrt(mse)
r2 = r2_score(y_valid,y_pred_blend1)

print('MSE : {:.3f}'.format(mse))
#print('RMSE: {:.3f}'.format(rmse))
#print('R2  : {:.3f}'.format(r2))

In [ ]:
# Cross-validation
kf = KFold(n_splits=5,random_state=None, shuffle=False)

params = [1,10,20]

# 5つのパートで検証されたメトリクスMSE
for rn in params:
    # 単一モデル
    regr = DecisionTreeRegressor(random_state=rn)
    result = -cross_val_score(regr,X,y, cv = kf, scoring = "neg_mean_squared_error")
    result_mean = np.mean(result)
    
    # print('CV_MSE:',result)
    print('CV_MSE_MEAN:{:.3f}'.format(result_mean),'MODEL:',str(regr))

In [ ]:
# ブレンディング 2
regr1 = DecisionTreeRegressor(random_state=1).fit(X_train,y_train)
regr2 = DecisionTreeRegressor(random_state=10).fit(X_train,y_train)
regr3 = DecisionTreeRegressor(random_state=20).fit(X_train,y_train)

y_pred1 = regr1.predict(X_valid)
y_pred2 = regr2.predict(X_valid)
y_pred3 = regr3.predict(X_valid)


# 推定値の平均値
y_pred_blend2 = np.mean([y_pred1,y_pred2,y_pred3],axis=0)
#print(y_pred)

# 評価
mse = mean_squared_error(y_valid,y_pred_blend2)
rmse = np.sqrt(mse)
r2 = r2_score(y_valid,y_pred_blend2)

print('MSE : {:.3f}'.format(mse))
#print('RMSE: {:.3f}'.format(rmse))
#print('R2  : {:.3f}'.format(r2))

In [ ]:
# 標準化
from sklearn.preprocessing import StandardScaler

std_mini_data = StandardScaler().fit_transform(mini_data)

In [ ]:
# 標準化後のデータセット可視化
sns.pairplot(pd.DataFrame(std_mini_data));

In [ ]:
# 説明変数X、目的変数y
X = std_mini_data[:,:-1]
y = std_mini_data[:,-1]

In [ ]:
# トレーニングデータと検証データに分割する（ホールドアウト法）
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = \
train_test_split(X,y,train_size=0.8,random_state=None)

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_valid:',X_valid.shape)
print('y_valid:',y_valid.shape)

In [ ]:
# 単一モデル
model = [LinearRegression(),
         SVR(),
         DecisionTreeRegressor()]


# Cross-validation
kf = KFold(n_splits=5,random_state=None, shuffle=False)

# 5つのパートで検証されたメトリクスMSE
for regr in model:
    result = -cross_val_score(regr,X,y, cv = kf, scoring = "neg_mean_squared_error")
    result_mean = np.mean(result)
    
    # print('CV_MSE:',result)
    print('CV_MSE_MEAN:{:.3f}'.format(result_mean),'MODEL:',str(regr))

In [ ]:
# ブレンディング 2
regr1 = LinearRegression().fit(X_train,y_train)
regr2 = SVR().fit(X_train,y_train)
regr3 = DecisionTreeRegressor().fit(X_train,y_train)

y_pred1 = regr1.predict(X_valid)
y_pred2 = regr2.predict(X_valid)
y_pred3 = regr3.predict(X_valid)


# 推定値の平均値
y_pred_blend3 = np.mean([y_pred1,y_pred2,y_pred3],axis=0)
#print(y_pred)

# 評価
mse = mean_squared_error(y_valid,y_pred_blend3)
rmse = np.sqrt(mse)
r2 = r2_score(y_valid,y_pred_blend3)

print('MSE : {:.3f}'.format(mse))
#print('RMSE: {:.3f}'.format(rmse))
#print('R2  : {:.3f}'.format(r2))

In [ ]:
# 問題2:バギングのスクラッチ実装

In [ ]:
# 説明変数X、目的変数y
X = log_min_data[['GrLivArea','YearBuilt']].values
y = log_min_data['SalePrice'].values

In [ ]:
X_train, X_valid, y_train, y_valid = \
    train_test_split(X,y,train_size=0.8,random_state=None)

In [ ]:
# 単一モデル
model = DecisionTreeRegressor().fit(X_train,y_train)
y_pred = model.predict(X_valid)

# 評価
mse = mean_squared_error(y_valid,y_pred)
print('MSE : {:.3f}'.format(mse))

In [ ]:
# バギング
n = 20
models = []

for i in range(n):
    X_bagging, X_, y_bagging, y_ = \
    train_test_split(X_train,y_train,train_size=0.2,shuffle=True)
    
    model = DecisionTreeRegressor()
    model.fit(X_bagging,y_bagging)
    models.append(model)

y_pred = np.zeros(len(X_valid))

for regr in models:
    pred = regr.predict(X_valid)
    y_pred = y_pred + pred
    
y_pred = y_pred/n

# 評価
mse = mean_squared_error(y_valid,y_pred)
print('MSE : {:.3f}'.format(mse))

In [ ]:
# 問題3:スタッキングのスクラッチ実装

In [ ]:
class Stacking():
    """
    スタッキングのClass
    Parameters
    ----------
    max_depth : int
      スタッキング可能な最大学習深度
    splits : int
      ブレンドデータ作成時のデータ分割数（CV分割数）
    models : dictionary
      学習モデル {key:n_depth, values:model} を渡す
    fit_models : list
      保存する学習済みモデルのリスト
    """
    def __init__(self, max_depth, splits, models):
        self.max_depth = max_depth
        self.n_splits = splits
        self.models = models
        self.fit_models = []
        
    def blending(self,X,y,m):
        """
        ブレンドデータ作成機能
        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            訓練データの特徴
        y : 次の形の ndarray, shape (n_samples,)
            学習データのラベル値
        m : class
            トレーニングモデルのインスタンス
        """
        self.y_blend = np.zeros(len(X))
        
        kf = KFold(n_splits=self.n_splits, shuffle=False)
        
        # CV
        for train_index, valid_index in kf.split(X):
            #print('KFold',count,'/',kf.get_n_splits())
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
            
            y_train = y_train.ravel()
            y_valid = y_valid.ravel()
            
            
            # トレーニングデータによるトレーニングモデル作成
            regr =  m
            regr.fit(X_train, y_train)
            # print(regr.predict(X_valid))
            self.fit_models.append(regr)
            
            # ブレンドデータ作成
            self.y_blend[valid_index] = regr.predict(X_valid)
    
    def fit_(self,X,y,depth):
        """
        この深さでブレンドデータを作成する
        Parameters
        ----------
        X : 次の形の ndarray、形状 (n_samples、n_features)
            訓練データの特徴
        y : 次の形の ndarray, shape (n_samples,)
            学習データのラベル値
        depth : int
            このステージの深さ
        """
        self.depth=depth
        
        # 最終学習モデル
        if self.depth == self.max_depth:
            self.model = self.models[self.depth]
            self.model.fit(X,y)
            return
        
        # この深さのトレーニングモデルを用意する
        models = self.models[self.depth]
        self.y_blending = np.zeros([len(X),len(models)])
        
        # この深さのトレーニングモデルでブレンドデータを作る
        # cvの空いている部分に入れている
        for i,mdl in enumerate(models):
            self.blending(X,y,mdl) 
            self.y_blending[:,i] = self.y_blend
        #　次のステージのためのX(blend_y)
        blend_y = self.y_blending
        
        # 再帰的
        self.bld = Stacking(self.max_depth, self.n_splits, self.models)
        self.bld.fit_(blend_y,y,depth+1)
        
    def predict_(self,X):
        if self.depth == self.max_depth:
            y_pred = self.model.predict(X)
            return y_pred
            
        else:
            tmp = 0
            self.y_pred = np.zeros(len(X))
            self.y_next = np.zeros([len(X),len(self.models[self.depth])])
            
            for mdl in self.fit_models:
                tmp+=1
                self.y_pred += mdl.predict(X)
                
                if tmp%self.n_splits == 0:
                    self.y_pred = self.y_pred/self.n_splits
                    self.y_next[:,int(tmp/self.n_splits)-1] = self.y_pred
                    self.y_pred = np.zeros(len(X))
                    
            y_pred = self.bld.predict_(self.y_next)
            
        return y_pred

In [ ]:
# 説明変数X、目的変数y
X = log_min_data[['GrLivArea','YearBuilt']].values
y = log_min_data['SalePrice'].values

In [ ]:
X_train, X_valid, y_train, y_valid = \
    train_test_split(X,y,train_size=0.8,random_state=None)

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_valid:',X_valid.shape)
print('y_valid:',y_valid.shape)

In [ ]:
model = {0:[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor()],
         1:[ARDRegression(),SGDRegressor(),DecisionTreeRegressor()],
         2:[HuberRegressor(),ARDRegression(),RandomForestRegressor()],
         3:LinearRegression()
         }

stk = Stacking(max_depth=3,splits=5,models=model)
stk.fit_(X_train,y_train,0)
y_pred = stk.predict_(X_valid)
print(y_pred)

# 評価
mse = mean_squared_error(y_valid,y_pred)
print('MSE : {:.3f}'.format(mse))

In [ ]:
# モデルサンプル
model =    {"LinearRegression": LinearRegression(),
            "Ridge": Ridge(),
            "Lasso": Lasso(),
            "ElasticNet": ElasticNet(), 
            "Polynomial_deg2": Pipeline([('poly', PolynomialFeatures(degree=2)),('linear', LinearRegression())]),
            "Polynomial_deg3": Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', LinearRegression())]),
            "Polynomial_deg4": Pipeline([('poly', PolynomialFeatures(degree=4)),('linear', LinearRegression())]),
            "Polynomial_deg5": Pipeline([('poly', PolynomialFeatures(degree=5)),('linear', LinearRegression())]),
            "KNeighborsRegressor": KNeighborsRegressor(n_neighbors=3),
            "DecisionTreeRegressor": DecisionTreeRegressor(),
            "RandomForestRegressor": RandomForestRegressor(),
            "SVR": SVR(kernel='rbf', C=1e3, gamma=0.1, epsilon=0.1),
            "GaussianProcessRegressor": GaussianProcessRegressor(),
            "SGDRegressor": SGDRegressor(),
            "MLPRegressor": MLPRegressor(hidden_layer_sizes=(10,10), max_iter=100, early_stopping=True, n_iter_no_change=5),
            "ExtraTreesRegressor": ExtraTreesRegressor(n_estimators=100), 
            "PLSRegression": PLSRegression(n_components=10),
            "PassiveAggressiveRegressor": PassiveAggressiveRegressor(max_iter=100, tol=1e-3),
            "TheilSenRegressor": TheilSenRegressor(random_state=0),
            "RANSACRegressor": RANSACRegressor(random_state=0),
            "HistGradientBoostingRegressor": HistGradientBoostingRegressor(),
            "AdaBoostRegressor": AdaBoostRegressor(random_state=0, n_estimators=100),
            "BaggingRegressor": BaggingRegressor(base_estimator=SVR(), n_estimators=10),
            "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
            "VotingRegressor": VotingRegressor([('lr', LinearRegression()), ('rf', RandomForestRegressor(n_estimators=10))]),
            "StackingRegressor": StackingRegressor(estimators=[('lr', RidgeCV()), ('svr', LinearSVR())], final_estimator=RandomForestRegressor(n_estimators=10)),
            "ARDRegression": ARDRegression(),
            "HuberRegressor": HuberRegressor(),
            }